In [1]:
import requests
import bs4
import pandas as pd
pd.set_option('max_colwidth', 400)

from IPython.display import HTML

In [6]:
def promo_shoes(HTML_display=False, sort_by='cut', sort_ascending=False):
    
    shoes_dict = {}
    base_url = 'https://sklep.sizeer.com' 
    urls = {
        'męskie': 'https://sklep.sizeer.com/meskie/buty/rozmiar-44__5,44,42__5,45,44-2___3?sort=price_asc',
        'damskie': 'https://sklep.sizeer.com/damskie/buty/rozmiar-39,38__5,38-2___3?sort=price_asc'
    }
    
    for url in urls.values():

        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.88 Safari/537.36'}
        res = requests.get(url, headers=headers)
        res.raise_for_status()

        sizeer_soup = bs4.BeautifulSoup(res.text, 'html.parser')
        items_list = sizeer_soup.select('.b-itemList_item')

        def parse_sizes(size):
            return float(size.replace('2/3', '.66').replace('1/3', '.33').replace(',', '.').replace(' ', ''))
        
        def parse_price(price):
            return float(price.split()[0].replace(',', '.'))

        for shoe in items_list:

            if shoe.get('data-ga-promo-info') == 'Cena promocyjna':      

                name = shoe.get('data-ga-name')
                color = shoe.get('data-ga-variant')
                price_regular = parse_price(shoe.select('.b-itemList_price')[0].getText())
                price_promo = parse_price(shoe.select('.b-itemList_price')[1].getText())
                cut = abs(int((price_promo / price_regular - 1) * 100))
                sizes = {parse_sizes(x.getText()) for x in shoe.select('.b-itemList_sizesItem')}
                url = base_url + shoe.select('.b-itemList_nameLink')[0].get('href')
                category = shoe.get('data-category')
                gender = 'MĘSKIE' if 'meskie' in url else 'DAMSKIE'

                if 'SERGIO TACCHINI' in name:            
                    continue
                elif ('Klapki' in category) & (cut < 50 ):
                    continue
                elif cut < 45:
                    continue
                elif (cut < 50) & (price_promo > 199.9):
                    continue
                elif (max(sizes) < 44) & ("CONVERSE" not in name) & ('meskie' in url):
                    continue

                shoes_dict[name + ' ' + color[0]] = {
                    'gender': gender,
                    'name': name + ' ' + color[0],
                    'price_regular': price_regular,
                    'price_promo': price_promo,
                    'cut': cut,
                    'sizes': sizes,
                    'category': category,
                    'url': url
                }
                
    if HTML_display:
        return HTML(pd.DataFrame(shoes_dict.values()).sort_values(sort_by, ascending=sort_ascending)
                    .to_html(render_links=True, escape=False))
    return pd.DataFrame(shoes_dict.values()).sort_values(sort_by, ascending=sort_ascending)
    

In [3]:
promo_shoes(HTML_display=True, sort_by='cut')

,gender,name,price_regular,price_promo,cut,sizes,category,url
7,DAMSKIE,FILA TROY SLIPPER WMN B,89.99,33.99,62,"{36.0, 37.0, 38.0, 39.0, 40.0}",Klapki i sandały,https://sklep.sizeer.com/fila-troy-slipper-wmn-damskie-klapki-i-sandaly-bialy-10103491fg
9,DAMSKIE,REEBOK CLUB C 85 PATENT BLACK WHITE C,399.99,152.99,61,{38.5},Sneakersy,https://sklep.sizeer.com/reebok-club-c-85-patent-black-white-damskie-sneakersy-czarny-bs9777
14,DAMSKIE,NIKE BLAZER LOW LE C,379.99,169.99,55,"{36.0, 37.5, 38.0, 38.5, 36.5, 39.0, 40.0}",Sneakersy,https://sklep.sizeer.com/nike-blazer-low-le-damskie-sneakersy-czarny-av9370-001
13,DAMSKIE,ADIDAS SUPERCOURT W R,379.99,169.99,55,"{36.66, 37.33, 38.66, 38.0, 39.33, 40.0, 40.66}",Sneakersy,https://sklep.sizeer.com/adidas-supercourt-w-damskie-sneakersy-rozowy-fv5470
1,MĘSKIE,CONVERSE PRO LEATHER GOLD STANDARD C,379.99,169.99,55,"{42.0, 42.5}",Sneakersy,https://sklep.sizeer.com/converse-pro-leather-gold-standard-meskie-sneakersy-czarny-167238c
0,MĘSKIE,DC STAG LITE S,279.99,127.49,54,"{41.0, 42.0, 43.0, 42.5, 46.0}",Sneakersy,https://sklep.sizeer.com/dc-stag-lite-meskie-sneakersy-szary-adys100597xksb
16,DAMSKIE,FILA RAY LOW WMN R,429.99,195.49,54,"{36.0, 37.0, 38.0, 39.0}",Sneakersy,https://sklep.sizeer.com/fila-ray-low-wmn-damskie-sneakersy-rozowy-101056271y
4,MĘSKIE,FILA ADRENALINE B,429.99,195.49,54,"{43.0, 44.0}",Sneakersy,https://sklep.sizeer.com/fila-adrenaline-meskie-sneakersy-bialy-101082792e
12,DAMSKIE,VANS UA OLD SKOOL Ż,339.99,169.99,50,"{36.0, 37.0, 38.0, 38.5, 36.5, 40.0, 40.5}",Trampki,https://sklep.sizeer.com/vans-ua-old-skool-damskie-trampki-zolty-vn0a4u3bwt71
10,DAMSKIE,VANS UA CLASSIC SLIP-ON P,299.99,152.99,49,"{36.5, 37.0, 38.5, 38.0, 39.0, 40.5}",Trampki,https://sklep.sizeer.com/vans-ua-classic-slip-on-damskie-trampki-pomaranczowy-vn0a4u38wt41
